# Oracle Machine Learning for Python - Data Selection and Manipulation
Oracle Machine Learning for Python (OML4Py), a component of the Oracle Advanced Analytics option to Oracle Database Enterprise Edition, makes the open source Python scripting language and environment ready for the enterprise and big data. Designed for problems involving both large and small volumes of data, Oracle Machine Learning for Python integrates Python with Oracle Database, allowing users to execute Python commands and scripts for statistical, machine learning, and graphical analyses on database tables and views using Python syntax. Many familiar Python functions are overloaded and translate Python functions into SQL for in-database execution, as well as new automated machine learning capabilities. 
![title](img/OML4P_icon.jpg)
In this notebook, we highlight various features of the transparency layer for data selection and manipulation. 

# Connect to Oracle Database
To use OML4Py, first import the package ***oml***. OML4Py supports a variety of connection specification options, including Oracle Wallet. Once connected to an Oracle Database that has OML4Py installed, invoking ***oml.isconnected*** returns true. 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import oml
oml.connect(user="pyquser",password="Cdoday19#Cdoday19#",dsn='(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=130.61.241.158)(PORT=1521))(CONNECT_DATA=(service_name=pdb2.sub12041412510.bdcevcn.oraclevcn.com)))')
oml.isconnected()

In [1]:
import oml
oml.connect(user="pyquser",password="pyquser#123pyquser#123",dsn='(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=130.61.60.94)(PORT=1521))(CONNECT_DATA=(service_name=pdb1.sub03010825490.devopsvcn.oraclevcn.com)))')
oml.isconnected()

True

Create a temporary OML DataFrame proxy object using ***push***. Then, select all rows with...
* the specified column names
* columns whose indices are in the range (1, 4)
* columns of oml.String data type

In [ ]:
PIZZA = oml.sync(view='V_PAYMENTS')
PIZZA.describe()

In [2]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()

x = pd.DataFrame(iris.data, columns = ['SEPAL_LENGTH','SEPAL_WIDTH',
                                       'PETAL_LENGTH','PETAL_WIDTH'])
y = pd.DataFrame(list(map(lambda x: {0: 'setosa', 1: 'versicolor', 
                          2:'virginica'}[x], iris.target)), 
                 columns = ['Species'])
z = pd.concat([x, y], axis=1)

IRIS_TMP = oml.push(z)

In [3]:
IRIS_projected1 = IRIS_TMP[:, ["SEPAL_LENGTH", "PETAL_LENGTH"]]
IRIS_projected1.head(3)

   SEPAL_LENGTH  PETAL_LENGTH
0           5.1           1.4
1           4.9           1.4
2           4.7           1.3

In [5]:
IRIS_projected2 = IRIS_TMP[:, 1:4]
IRIS_projected2.head(3)
oml.create(z,'IRIS')

     SEPAL_LENGTH  SEPAL_WIDTH  PETAL_LENGTH  PETAL_WIDTH    Species
0             5.1          3.5           1.4          0.2     setosa
1             4.9          3.0           1.4          0.2     setosa
2             4.7          3.2           1.3          0.2     setosa
3             4.6          3.1           1.5          0.2     setosa
4             5.0          3.6           1.4          0.2     setosa
5             5.4          3.9           1.7          0.4     setosa
6             4.6          3.4           1.4          0.3     setosa
7             5.0          3.4           1.5          0.2     setosa
8             4.4          2.9           1.4          0.2     setosa
9             4.9          3.1           1.5          0.1     setosa
10            5.4          3.7           1.5          0.2     setosa
11            4.8          3.4           1.6          0.2     setosa
12            4.8          3.0           1.4          0.1     setosa
13            4.3          3.0    

In [ ]:
IRIS_projected3 = IRIS_TMP.select_types(include=[oml.String])
IRIS_projected3.head(3)

In [6]:
def return_df(num, scale):
    import pandas as pd
    id = list(range(0,int(num)))
    res = [i/scale for i in id]
    return pd.DataFrame({"ID":id, "RES":res})
res = oml.do_eval(func=return_df, func_value=pd.DataFrame({"ID":[0],"RES":[0]}), scale=100, num=10)
res

   ID   RES
0   0  0.00
1   1  0.01
2   2  0.02
3   3  0.03
4   4  0.04
5   5  0.05
6   6  0.06
7   7  0.07
8   8  0.08
9   9  0.09

Let's explore various ways of filtering data in-database. 
* select sepal length and petal length where petal length is less than 1.5
* select all rows in which petal length is less than 1.5 or sepal length is 5.0, using the AND and OR conditions in filtering
* select all rows in which petal length is less than 1.5 and sepal length is larger than 5.0

In [ ]:
IRIS_filtered1 = IRIS_TMP[IRIS_TMP["PETAL_LENGTH"] < 1.5, 
                                  ["SEPAL_LENGTH", "PETAL_LENGTH"]]
print("Length: ", len(IRIS_filtered1))
IRIS_filtered1.head(3)

In [ ]:
IRIS_filtered2 = IRIS_TMP[(IRIS_TMP["PETAL_LENGTH"] < 1.5) | 
                          (IRIS_TMP["SEPAL_LENGTH"] == 5.0), :]
print("Length: ", len(IRIS_filtered2))
IRIS_filtered2.head(3)

In [ ]:
IRIS_filtered3 = IRIS_TMP[(IRIS_TMP["PETAL_LENGTH"] < 1.5) & 
                          (IRIS_TMP["SEPAL_LENGTH"] > 5.0), :]
print("Length: ", len(IRIS_filtered3))
IRIS_filtered3.head()

# Using Pandas DataFrame objects
Let's explore operations involving Pandas DataFrames. 

We'll start by creating a temporary table from a Pandas DataFrame and use the ***append*** function. First, append an oml.Float series object to another, then append an oml.DataFrame object to another.

In [ ]:
my_df = pd.DataFrame({"id" : [1, 2, 3, 4, 5],
                      "val" : ["a", "b", "c", "d", "e"],
                      "ch" : ["p", "q", "r", "a", "b"],
                      "num" : [4, 3, 6.7, 7.2, 5]})
MY_DF = oml.push(my_df)

print (my_df.dtypes)

num1 = MY_DF['id']
num2 = MY_DF['num']
num1.append(num2)

In [ ]:
x = MY_DF[['id', 'val']] 
y = MY_DF[['num', 'ch']] 

print(x.dtypes)
print(y.dtypes)

x.append(y)


Next, let's try a few examples using the ***concat*** function.
* create two oml.DataFrame objects and combine the objects column-wise
* create an oml.Float object with the rounded exponential of two times the values in the num column of the oml_frame object, then concatenate it with the oml.DataFrame object y using a new column name
* concatenate object x with multiple objects and turn on automatic name conflict resolution
* concatenate multiple OML data objects and perform customized renaming

In [ ]:
from collections import OrderedDict

x = MY_DF[['id', 'val']]
y = MY_DF[['num', 'ch']]
x.concat(y)

In [ ]:
w = (MY_DF['num']*2).exp().round(decimals=2)
y.concat({'round(exp(2*num))':w})

In [ ]:
z = MY_DF[:,'id']
x.concat([z, w, y], auto_name=True)

In [ ]:
x.concat(OrderedDict([('ID',z), ('round(exp(2*num))',w), ('New_',y)]))

Next, we explore merging data. Perform:
* cross join
* left outer join
* right outer join

In [ ]:
x = MY_DF[['id', 'val']]
y = MY_DF[['num', 'ch']]

z = x.merge(y)
z

In [ ]:
x.head(4).merge(other=MY_DF[['id', 'num']], on="id", suffixes=['.l','.r'])

In [ ]:
x.merge(other=y, left_on="id", right_on="num", how="right")

Next, try the _drop_ functonality with ***dropna***, ***drop_duplicates***, and ***drop***:
* drop rows with any missing values
* drop rows in which all column values are missing
* drop rows in which any numeric column values are missing
* drop duplicate rows
* drop rows that have the same value in column 'string1' and 'string2'
* drop column 'string2'

In [ ]:
my_df2 = pd.DataFrame({'numeric': [1, 1.4, -4, -4, 5.432, None, None],
                   'string1' : [None, None, 'a', 'a', 'a', 'b', None],
                   'string2': ['x', None, 'z', 'z', 'z', 'x', None]})
MY_DF2 = oml.push(my_df2, dbtypes = {'numeric': 'BINARY_DOUBLE',
                                     'string1':'CHAR(1)', 
                                     'string2':'CHAR(1)'})
MY_DF2    

In [ ]:
MY_DF2.dropna(how='any')

In [ ]:
MY_DF2.dropna(how='all')

In [ ]:
MY_DF2.dropna(how='any', subset=['numeric'])

In [ ]:
MY_DF2.drop_duplicates()

In [ ]:
MY_DF2.drop_duplicates(subset=['string1', 'string2'])

In [ ]:
MY_DF2.drop('string2')

Next, we explore the ***split*** and ***KFold*** function using the _digits_ data set. After creating an OML DataFrame proxy object for the _digits_ data set, 
* sample 20% and 80% of the data
* split the data into four sets
* perform stratification on the target column
* verify that the stratified sampling generates splits in which all of the different categories of digits (digits 0~9) are present in each split
* hash on the target column
* verify that the different categories of digits (digits 0~9) are present in only one of the splits generated by hashing on the category column
* split the data randomly into 4 consecutive folds

In [ ]:
import pandas as pd
from sklearn.datasets import load_digits

digits = load_digits()
pd_digits = pd.DataFrame(digits.data,
                         columns=['IMG'+str(i) for i in
                         range(digits['data'].shape[1])])
pd_digits = pd.concat([pd_digits,
                       pd.Series(digits.target,
                                  name = 'target')],
                                  axis = 1)
DIGITS = oml.push(pd_digits)
print("Shape: ", DIGITS.shape)

In [ ]:
splits = DIGITS.split(ratio=(.2, .8), use_hash = False)
print("Split lengths: ", [len(split) for split in splits])

In [ ]:
splits = DIGITS.split(ratio = (.25, .25, .25, .25), use_hash = False)
print("Split lengths: ", [len(split) for split in splits])

In [ ]:
splits = DIGITS.split(strata_cols=['target'])
print("Split lengths: ", [split.shape for split in splits])

In [ ]:
print("Verify values: ", [split['target'].drop_duplicates().sort_values().pull()
for split in splits])

In [ ]:
splits = DIGITS.split(hash_cols=['target'])
print("Split lengths: ", [split.shape for split in splits])

In [ ]:
print("Verify values: ", [split['target'].drop_duplicates().sort_values().pull()
for split in splits])

In [ ]:
folds = DIGITS.KFold(n_splits=4)
print("Split lengths: ", [(len(fold[0]), len(fold[1])) for fold in folds])

Next, we explore the ***corr*** function. We construct a trivial, highly correlated DataFrame. First, verify that the correlation between column A and B is 1.

In [ ]:
import pandas as pd

my_df2 = pd.DataFrame({'A': range(4), 'B': [2*i for i in range(4)]})
MY_DF2 = oml.push(my_df2)
MY_DF2

In [ ]:
MY_DF2.corr()

Make a few changes to the data to alter the correlation. 

In [ ]:
my_df3 = my_df2

# Change a value to test the change in the computed correlation result
my_df3.loc[2, 'A'] = 1.5

# Change an entry to NaN (not a number) to test the 'skipna' parameter in corr
my_df3.loc[1, 'B'] = None

# Push my_df3 to Oracle Database using the floating point column type 
# because NaNs cannot be used in Oracle numbers
MY_DF3 = oml.push(my_df3, oranumber=False)
MY_DF3

By default, 'skipna' is True.

In [ ]:
MY_DF3.corr()

In [ ]:
MY_DF3.corr(skipna=False)

Next, we explore the ***crosstab*** and ***pivot*** functions.

In [ ]:
my_df4 = pd.DataFrame({
     'GENDER': ['M', 'M', 'F', 'M', 'F', 'M', 'F', 'F', None, 'F', 'M', 'F'],
     'HAND': ['L', 'R', 'R', 'L', 'R', None, 'L', 'R', 'R', 'R', 'R', 'R'],
     'SPEED': [40.5, 30.4, 60.8, 51.2, 54, 29.3, 34.1, 39.6, 46.4, 12, 25.3, 37.5],
     'ACCURACY': [.92, .94, .87, .9, .85, .97, .96, .93, .89, .84, .91, .95]
    })
MY_DF4 = oml.push(my_df4)

Find the categories that the most entries belonged to.

In [ ]:
MY_DF4.crosstab('GENDER', 'HAND').sort_values('count', ascending=False)

For each gender value and across all entries, find the ratio of entries with different hand values.

In [ ]:
MY_DF4.crosstab('GENDER', 'HAND', pivot = True, margins = True, normalize = 0)

Find the mean speed across all gender and hand combinations.

In [ ]:
MY_DF4.pivot_table('GENDER', 'HAND', 'SPEED')

Find the median accuracy and speed for every gender and hand combination.

In [ ]:
MY_DF4.pivot_table('GENDER', 'HAND', aggfunc = oml.DataFrame.median)

Find the max and min speeds for every gender and hand combination and across all combinations.

In [ ]:
MY_DF4.pivot_table('GENDER', 'HAND', 'SPEED', 
               aggfunc = [oml.DataFrame.max, oml.DataFrame.min],
               margins = True)

Next, we explore creating new columns using a simple shopping cart data set. Note, that using the transparency layer, all these computations occur in-database. Neither the input nor the result needs to be brought to the client Python engine. 

In [ ]:
shopping_cart = pd.DataFrame({
  'Item_name': ['paper_towel', 'ground_pork', 'tofu', 'eggs',
                'pork_loin', 'whole_milk', 'egg_custard'],
  'Item_type': ['grocery', 'meat', 'grocery', 'dairy', 'meat',
                'dairy', 'bakery'],
  'Quantity': [1, 2.6, 4, 1, 1.9, 1, 1],
  'Unit_price': [1.19, 2.79, 0.99, 2.49, 3.19, 2.5, 3.99]
  })
CART = oml.push(shopping_cart)
CART

Add a column 'Price' multiplying 'Quantity' with 'Unit_price', rounded to 2 decimal places.

In [ ]:
price = CART['Quantity']*(CART['Unit_price'])
print("Type: ", type(price))
CART = CART.concat({'Price': price.round(2)})
CART

Count the pattern 'egg' in the 'Item_name' column.

In [ ]:
egg_pattern = CART['Item_name'].count_pattern('egg')
print("Type: ", type(egg_pattern))
CART.concat({'Egg_pattern': egg_pattern})

Find the start index of substring 'pork' in the 'Item_name' column.

In [ ]:
pork_startInd = CART['Item_name'].find('pork')
print("Type: ", type(pork_startInd))
CART.concat({'Pork_startInd': pork_startInd})

Check whether items are of grocery category.

In [ ]:
is_grocery=CART['Item_type']=='grocery'
print("Type: ", type(is_grocery))
CART.concat({'Is_grocery': is_grocery})

Calculate the length of item names.

In [ ]:
name_length=CART['Item_name'].len()
print("Type: ", type(name_length))
CART.concat({'Name_length': name_length})

Use a few other function to modify the Price column. 

In [ ]:
print("Ceiling: ", CART['Price'].ceil())
print("Floor: ", CART['Price'].floor())
print("Exponential: ", CART['Price'].exp().round(3))
print("logarithm: ", CART['Price'].log().round(3))
print("Square Root: ", CART['Price'].sqrt().round(3))

Next, using a modified version of the _iris_ data set, explore ***sort_values*** and ***tail*** functions. 

We modify the dataset by replacing a few entries with NaNs to test how the _na_position_ parameter works in the ***sort_values*** method.

In [ ]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()

x = pd.DataFrame(iris.data, columns = ['Sepal_Length', 'Sepal_Width',
                                       'Petal_Length', 'Petal_Width'])
y = pd.DataFrame(list(map(lambda x: {0: 'setosa', 1: 'versicolor',
                                     2:'virginica'}[x], iris.target)), 
                 columns = ['Species'])
                   
x['Sepal_Width'].replace({3.5: None}, inplace=True)
x['Petal_Length'].replace({1.5: None}, inplace=True)
x['Petal_Width'].replace({2.3: None}, inplace=True)

iris_df = pd.concat([x, y], axis=1)
IRIS2 = oml.push(iris_df)

Sort the data set first by Sepal_Length then by Sepal_Width in descending order and display the first 5 rows of the sorted result.

In [ ]:
IRIS2.sort_values(by = ['Sepal_Length', 'Sepal_Width'],
                  ascending=False).head()

Display the last 5 rows of the data set.

In [ ]:
IRIS2.tail()

Sort the last 5 rows of the iris data set first by Petal_Length then by Petal_Width. By default, rows with NaNs are placed after the other rows when the sort keys are the same.

In [ ]:
IRIS2.tail().sort_values(by = ['Petal_Length', 'Petal_Width'])

Sort the last 5 rows of the IRIS data set first by Petal_Length and then by Petal_Width. When the values in these two columns are the same, place the row with a NaN before the other row.

In [ ]:
IRIS2.tail().sort_values(by = ['Petal_Length', 'Petal_Width'], 
                                 na_position = 'first')

Next, we explore column creation/renaming, describing data statistics, and computing a variety of statistics. 

In [ ]:
my_df4 = pd.DataFrame({'numeric_col': [1, 1.4, -4, 3.145, 5, None],
                       'string_col' : [None, None, 'a', 'a', 'a', 'b'],
                       'bytes_col' : [b'a', b'b', b'c', b'c', b'd', b'e']})

MY_DF4 = oml.push(my_df4, dbtypes = {'numeric_col': 'BINARY_DOUBLE',
                                     'string_col':'CHAR(1)', 
                                     'bytes_col':'RAW(1)'})
MY_DF4

Combine a Boolean column with MY_DF4.

In [ ]:
oml_bool = MY_DF4['numeric_col'] > 3

MY_DF4 = MY_DF4.concat(oml_bool)
MY_DF4.rename({'COL4':'boolean_col'})

Use the ***describe*** function to compute a variety of statistics on all columns. Then, exclude the Float columns. 

In [ ]:
MY_DF4.describe(include='all')

In [ ]:
MY_DF4.describe(exclude=[oml.Float])

Compute the sum of values in each Float or Boolean column.

In [ ]:
MY_DF4.sum()

Find the cumulative sum of values in each Float or Boolean column after MY_DF4 is sorted by the bytes column in descending order. Then, compute skew, median, and kurtosis for float columns. 

In [ ]:
MY_DF4.cumsum(by = 'bytes_col', ascending = False)

In [ ]:
MY_DF4.skew()

In [ ]:
MY_DF4.median()

In [ ]:
MY_DF4.kurtosis()

Next, explore the ***boxplot*** and ***hist*** functions using the _wine_ data set. The statistics supporting these plots are computed in-database, so transfer of data or memory limitations are avoided. 

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine

wine = load_wine()
wine_data = pd.DataFrame(wine.data, columns = wine.feature_names)
WINE = oml.push(wine_data)

oml.graphics.boxplot(WINE[:,8:12], showmeans=True, 
                     meanline=True, patch_artist=True, 
                     labels=WINE.columns[8:12])
plt.title('Distribution of Wine Attributes')
plt.show()

In [ ]:
oml.graphics.hist(WINE['proline'], bins=10, color='red',
                  linestyle='solid', edgecolor='black')
plt.title('Proline content in Wine')
plt.xlabel('proline content')
plt.ylabel('# of wine instances')
plt.show()

In [7]:
oml.disconnect()



<img src="img/Oracle-sm.jpg">